In [0]:
%run "./config/v5/configurations-asc1"

In [0]:
%run "./config/v5/common"

data_source_id,schema_version,schema_ddl,logic_mapping
b31d684c-d84b-44fa-9873-47c561542df9,6,"`Line` STRING, `Station` STRING, `Part Number` STRING, `Database Code` STRING, `Serial Number` STRING, `Time` STRING, `Gun` STRING, `Job` STRING, `Pass` STRING, `Torque` STRING, `Ang (deg.)` STRING","{""line"":[""Line""], ""station_config"":[""Line"", ""Station""], ""sensor_config"":[""Line"", ""Station"", ""Gun"", ""Job""], ""part_number"": [""Part_Number""], ""serial_number"": [""Serial_Number""], ""measurement"":[""Torque"", ""Ang__deg__""], ""measured_time"":""Time""}"
d21ae4b8-51d8-4990-b9da-91c5cfc67927,7,"`serial_number` STRING, `process_name` STRING, `process_attribute_1` STRING, `process_attribute_2` STRING, `process_attribute_3` STRING, `process_attribute_4` STRING, `process_attribute_5` STRING, `data_element_name` STRING, `data_element_attribute_1` STRING, `data_element_attribute_2` STRING, `data_element_attribute_3` STRING, `data_element_attribute_4` STRING, `data_element_attribute_5` STRING, `location_name` STRING, `parent_location_name` STRING, `part_number` STRING, `data_value` STRING, `created` STRING, `trace` STRING, `test` STRING","{""line"":[""parent_location_name""], ""station_config"":[""location_name""], ""sensor_config"":[""parent_location_name"", ""location_name"", ""process_name"", ""process_attribute_1"", ""process_attribute_2"", ""process_attribute_3"", ""process_attribute_4"", ""process_attribute_5"", ""data_element_name"", ""data_element_attribute_1"", ""data_element_attribute_2"", ""data_element_attribute_3"", ""data_element_attribute_4"", ""data_element_attribute_5""], ""part_number"": [""part_number""], ""serial_number"": [""serial_number""], ""measurement"":[""data_value""], ""measured_time"":""created""}"
86494c9c-143c-2e85-3b50-2021de22c403,9,"`serial_number` STRING, `process_name` STRING, `process_attribute_1` STRING, `process_attribute_2` STRING, `process_attribute_3` STRING, `process_attribute_4` STRING, `process_attribute_5` STRING, `data_element_name` STRING, `data_element_attribute_1` STRING, `data_element_attribute_2` STRING, `data_element_attribute_3` STRING, `data_element_attribute_4` STRING, `data_element_attribute_5` STRING, `location_name` STRING, `parent_location_name` STRING, `part_number` STRING, `data_value` STRING, `created` STRING, `trace` STRING, `test` STRING","{""line"":[""parent_location_name""], ""station_config"":[""location_name""], ""sensor_config"":[""parent_location_name"", ""location_name"", ""process_name"", ""process_attribute_1"", ""process_attribute_2"", ""process_attribute_3"", ""process_attribute_4"", ""process_attribute_5"", ""data_element_name"", ""data_element_attribute_1"", ""data_element_attribute_2"", ""data_element_attribute_3"", ""data_element_attribute_4"", ""data_element_attribute_5""], ""part_number"": [""part_number""], ""serial_number"": [""serial_number""], ""measurement"":[""data_value""], ""measured_time"":""created""}"
2665e070-cbde-ac9f-d6d0-228d82bc75ac,8,"`serial_number` STRING, `process_name` STRING, `process_attribute_1` STRING, `process_attribute_2` STRING, `process_attribute_3` STRING, `process_attribute_4` STRING, `process_attribute_5` STRING, `data_element_name` STRING, `data_element_attribute_1` STRING, `data_element_attribute_2` STRING, `data_element_attribute_3` STRING, `data_element_attribute_4` STRING, `data_element_attribute_5` STRING, `location_name` STRING, `parent_location_name` STRING, `part_number` STRING, `data_value` STRING, `created` STRING, `trace` STRING, `test` STRING","{""line"":[""parent_location_name""], ""station_config"":[""location_name""], ""sensor_config"":[""parent_location_name"", ""location_name"", ""process_name"", ""process_attribute_1"", ""process_attribute_2"", ""process_attribute_3"", ""process_attribute_4"", ""process_attribute_5"", ""data_element_name"", ""data_element_attribute_1"", ""data_element_attribute_2"", ""data_element_attribute_3"", ""data_element_attribute_4"", ""data_element_attribute_5""], ""

data_source_id,client,location,line,source_type,folder_location
b31d684c-d84b-44fa-9873-47c561542df9,Borg Warner,XYZ,L14,csv,server://path
d21ae4b8-51d8-4990-b9da-91c5cfc67927,Dana,Birmingham,ASC1,csv,server://path
86494c9c-143c-2e85-3b50-2021de22c403,Dana,Dry Ridge,700_line,csv,server://path
2665e070-cbde-ac9f-d6d0-228d82bc75ac,Dana,Columbia,Loop_1,csv,server://path


In [0]:
#**********************************************************************************************************************
# RESET STREAMING (for testing and dev only)
# use this code block to clear streaming checkpoints and output folders 
#**********************************************************************************************************************

#dbutils.fs.rm(checkpointPath, True)
#dbutils.fs.rm(pivotRootDir, True)
#dbutils.fs.rm(cubeRunRootDir, True)


#**********************************************************************************************************************
# RESET PIVOT SCHEMA TABLE (for testing and dev only)
# use this code block to create an empty pivot schema table 
#**********************************************************************************************************************

### To initialize pivot schema table (run only once)
pivot_schema_df = spark.createDataFrame(data=[],schema=T._parse_datatype_string(pivot_schema_table_ddl))
# pivot_schema_df.write.parquet(pivotSchemaDir)

### Load pivot schema table
# pivot_schema_df = spark.read.parquet(pivotSchemaDir)
pivot_schema_df.registerTempTable("pivot_schema_table")
display(spark.sql("select * from pivot_schema_table"))

dataSourceId,pivotSchemaVersion,pivotIndexDdl,featureList,updated_on


In [0]:
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark import Row
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.types import *
import json

#**********************************************************************************************************************
# HOUSE KEEPING
#**********************************************************************************************************************

# set the basic spark configurations
## the number of shuffle partitions should be set as multiples of the total work cores
spark.conf.set("spark.sql.shuffle.partitions", 32)
## enables the schema merging effect of the delta table
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", True)
## to guaranteed successful read of the schema parquet file
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")

# load configuration and other variables through Notebook workflow
#dbutils.notebook.run("./configurations", 60)


#**********************************************************************************************************************
# DATA INGESTION: read stream from Kafka topic and apply ingress data schema
#**********************************************************************************************************************

input_df = (
  spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", conf['bootstrap.servers'])
  .option("kafka.security.protocol", "SASL_SSL")
  .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(conf['sasl.username'], conf['sasl.password']))
  .option("kafka.ssl.endpoint.identification.algorithm", "https")
  .option("kafka.sasl.mechanism", "PLAIN")
  .option("subscribe", conf['topic'])
  .option("startingOffsets", conf['startingOffsets'])
  .option("failOnDataLoss", "false")
  .option("maxOffsetsPerTrigger", "4096")
  .load()
  .select(col("value").cast("STRING"), col("key").cast("STRING")) 
  .select(from_json("value", ingress_schema).alias("ingress_json"))
)


#**********************************************************************************************************************
# TRANSFORM STEP 1/5: extract data source id, schema version, and measurement dataframe from ingested Kafka record.
# The column headers of the measurement dataframe are cleaned to remove illegal characters.
#**********************************************************************************************************************

def replace_special_char(column_name):
  """ replace all non-alphanumeric characters with an underscore. """
  return regexp_replace(column_name, r'[^a-zA-Z0-9]', '_')

formatted_df = (input_df
                .select(col("ingress_json.dataSourceId").alias("dataSourceId"), 
                        col("ingress_json.schemaVersion").alias("schemaVersion"), explode("ingress_json.rows").alias("records"))
                .withColumn("headers", transform(f.map_keys("records"), replace_special_char))
                .withColumn("data", map_values("records"))
               .withColumn("mapped_fields",  map_from_arrays("headers", "data"))
               .select("dataSourceId", "schemaVersion", "mapped_fields")
              )


#display(formatted_df)

In [0]:
#**********************************************************************************************************************
# TRANSFORM STEP 2/5: apply business logic mapping to source data columns
# NOTE: the business-mapping config json string is read from the business-schema table and applied in realtime to
# the data records. Records with different data source id and schema version will have different schemas applied.
#**********************************************************************************************************************

def mapLogicalFields(fields, logic_mapping):
  """
  udf to map data fields according to business logic mapping config
  
  Parameters
    ----------
    fields : array<map>
        An array of maps with column header as keys and data as values. One array corresponds to one row of data 
    logic_mapping : string
        The json string containing business mapping config

    Returns
    -------
    logic_mapped_dict : dict
        A dict with the logic entites (line, station, sensor, part number, serial number, timestamp) as keys
        and data mapped from the incoming data records as values
  """
  
  mappings = json.loads(logic_mapping)
  ret, measurement_dict = {}, {}
  for attr in mappings:
    if attr=='measured_time':
      ret[config_col_mapping[attr]]=fields[mappings[attr]]
    elif attr=='measurement':
      for m in mappings[attr]:
        measurement_dict[m]=fields[m]
    else:
      ret[config_col_mapping[attr]] = '_'.join([fields[c] for c in mappings[attr]])
  ret['measurements']=measurement_dict
  return ret

# register the udf
logicMapUDF = udf(lambda x, y: mapLogicalFields(x, y), mapped_schema)


mapped_df = (formatted_df
             .join(business_schema_df, 
                   (formatted_df["dataSourceId"]==business_schema_df["data_source_id"]) & 
                   (formatted_df["schemaVersion"]==business_schema_df["schema_version"])    # EXCEPTION HANDLING for unknown source/schemaVersion
                  )
             .withColumn("logic_mapped", logicMapUDF(col("mapped_fields"), col("logic_mapping")))
             .select("dataSourceId", "schemaVersion", "logic_mapped")
             .filter("logic_mapped.serial_number!=''")     # DISCUSSION: how to handle records with missing serial numbers
            )

#display(mapped_df)

In [0]:
## For dev & test only: create new sensor to uuid mapping table
# sensorDf = (mapped_df.select("logic_mapped.sensor", "logic_mapped.line", "logic_mapped.station")
#             .distinct()
#             .selectExpr('*', "uuid() as sensor_uuid")
#            )
# display(sensorDf)

In [0]:
# dbutils.fs.rm('/FileStore/tables/sensor_mapping_all_lines.csv', True)

In [0]:
#**********************************************************************************************************************
# TRANSFORM STEP 3/5: replace sensor name with uuid, then return a dataframe with business entities as column headers, 
# a list of feature names, and a kev-value list of feature-name and data
#**********************************************************************************************************************
uuid_df = (mapped_df
           .join(sensorDf.select("sensor", "sensor_uuid"), mapped_df["logic_mapped.sensor"]==sensorDf["sensor"], "left")
           .withColumn("feature_names", transform(map_keys("logic_mapped.measurements"), lambda x: concat(col('sensor_uuid'), lit('_'), x)))
           .withColumn("feature_json", transform(arrays_zip("feature_names", map_values("logic_mapped.measurements")),
                                                 lambda x: concat(lit('"'), x["feature_names"], lit('":"'), x['1'], lit('"'))
                                                ))
           .select("dataSourceId", "schemaVersion",
                   col("logic_mapped.line").alias('line'), col("logic_mapped.part_number").alias('part_number'), 
                   col("logic_mapped.serial_number").alias('serial_number'), col("logic_mapped.station").alias('station'), 
                   col("logic_mapped.timestamp").alias('timestamp'), "feature_json", "feature_names"
                  )
          )


#**********************************************************************************************************************
# TRANSFORM STEP 4/5: collapse rows with same timestamp for each serial number into a single row, and create a
# 'measurement_date' column to be used as partition column
#**********************************************************************************************************************
collapsed_df = (uuid_df
                 .groupBy("dataSourceId", "schemaVersion", "line", "part_number","serial_number", "timestamp")
                 .agg(collect_set('station').alias("station_list"), 
                      concat(lit("{"), concat_ws(", ", array_distinct(flatten(collect_list("feature_json")))), lit("}")).alias("feature_json"),
                      array_distinct(flatten(collect_list("feature_names"))).alias("feature_names")
                     )
                 .withColumn("measurement_date", to_date("timestamp"))
                )


#**********************************************************************************************************************
# TRANSFORM STEP 5/5: join with source_meta_data_table to get full name of the line, which will decide which cube
# the transformed data is dropped into
#**********************************************************************************************************************
pivot_prep_df = (collapsed_df
                 .join(
                   data_source_meta_df.select("data_source_id", concat_ws("_", "client", "location", "line").alias("line_full_name")), 
                   collapsed_df["dataSourceId"]==data_source_meta_df["data_source_id"])
                 .withColumn("line_full_name", replace_special_char("line_full_name"))
                 .drop("data_source_id")
                )


#display(pivot_prep_df)

In [0]:
### helper functions needed to dynamically generate and update pivot table schema

def getCurrPivotSchemaLocal(dataSourceId):
  """
  returns the current features list from the pivot-schama-table and the computed next pivot schema version number 
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The data source schema version of the processed records

    Returns
    -------
    features_list : set
        A list of current features
    next_pivot_schema_version : int
        Version number of the next pivot schema to be inserted
  """
  
  res = spark.sql(f'select featureList, pivotSchemaVersion from pivot_schema_table \
                   where dataSourceId="{dataSourceId}" and pivotSchemaVersion= \
                   (select max(pivotSchemaVersion) from pivot_schema_table where dataSourceId="{dataSourceId}")').first()
  if res is None:
    return set(), 1
  return set(res[0].split(',')), res[1]+1


### TO BE IMPLEMENTED
def getCurrPivotSchemaPostgres(dataSourceId, schemaVersion):
  """ returns the current features list from a postgreSQL table and the computed next pivot schema version number """
  pass


def updatePivotSchemaLocal(data_source_id, pivot_idx_ddl, pivot_schema_version, feature_list):
  """
  updates the features list and pivot-schema version in the pivot-schama-table 
  and returns the next pivot-schema version number 
  
  Parameters
    ----------
    data_source_id : string
        The ID string of the data source 
    schema_version : string
        The data source schema version of the processed records
    pivot_idx_ddl : string
        The ddl string of index columns in the pivot-schema-table
    pivot_schema_version : int
        Pivot-schema version number of the new schema to be inserted
    feature_list : set
        A list of current features in the new schema

    Returns
    -------
    next_pivot_schema_version : int
        Version number of the next pivot schema to be inserted
  """
  
  feature_list_string = ','.join(feature_list)
  original_df = spark.sql("select * from pivot_schema_table")
  append_df = (spark.createDataFrame(data=[(data_source_id, pivot_schema_version, pivot_idx_ddl, 
                                                  feature_list_string, datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:23])])
                    .toDF(*original_df.columns)
              )
  pivot_schema_df = original_df.union(append_df)
  pivot_schema_df.registerTempTable("pivot_schema_table")
  append_df.write.mode('append').parquet(pivotSchemaDir)
  return pivot_schema_version+1


### TO BE IMPLEMENTED
def update_pivot_schema_postgres(dataSourceId, schemaVersion):
  """
  updates the features list and pivot-schema version in the postgreSQL table 
  and returns the next pivot-schema version number 
  """
  pass


def getDataSourceAndSchemaVersion(df):
  """
  returns the first data source id and schema version from a given dataframe 
  
  Parameters
    ----------
    df : dataframe
        A dataframe containing 'dataSourceId' and 'schemaVersion' columns

    Returns
    -------
    dataSourceId : string
        Data source ID of the data record
    SchemaVersion : int 
        Schema version of the data record
  """
  res = df.select("dataSourceId", "schemaVersion").first()
  if res is None:
    return "", ""
  return res[0], res[1]


def getFeatureListFromDF(df):
  """
  returns the list of unique feature names from a given dataframe 
  
  Parameters
    ----------
    df : dataframe
        A dataframe containing 'feature_names' column

    Returns
    -------
    features_list : list
        A list of unique feature names
  """
  
  res = df.agg(array_distinct(flatten(collect_list("feature_names")))).first()
  if res is None:
    return []
  return res[0]


def getFeatureListLocal(dataSourceId):
  """
  returns the current features list from the pivot-schama-table 
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The schema version of the data source

    Returns
    -------
    features_list : set
        A list of current features
  """
  res = spark.sql(f'select featureList from pivot_schema_table \
                   where dataSourceId="{dataSourceId}" and pivotSchemaVersion= \
                   (select max(pivotSchemaVersion) from pivot_schema_table where dataSourceId="{dataSourceId}")').first()
  if res is None:
    return set()
  return set(res[0].split(','))


def initialize_delta_sink(line):
  """ create an empty delta table in the cube directory for a given line """
  (spark
 .createDataFrame([], schema=T._parse_datatype_string(pivot_idx_ddl))
 .write
 .option("mergeSchema", "true")
 .partitionBy("measurement_date") 
 .format("delta")
 .mode("append")
 .save(pivotRootDir + f"{line}/"))
  
  
def load_features_for_line(data_source_id):
  """
  returns the current features list, next pivot schema version, and current features ddl string for
  given data_source_id and schema_version
  
  Parameters
    ----------
    dataSourceId : string
        The ID string of the data source 
    schemaVersion : string
        The schema version of the data source

    Returns
    -------
    curr_features_list : set
        A list of current features
    next_pivot_schema_version : int
        The next pivot schema version
    curr_feature_ddl : string
        A ddl string of the current features
  """
  curr_feature_list, next_pivot_schema_version = getCurrPivotSchemaLocal(data_source_id)
  curr_feature_ddl = "`" + '` STRING, `'.join(sorted(curr_feature_list)) + "` STRING"
  return curr_feature_list, next_pivot_schema_version, curr_feature_ddl


def process_batch_for_line(line, line_df, data_source_id):
  """
  Process batch dataframe for a given line, create a new version of pivot schema if new features are seen, 
  and updates the delta table sink by performing 'upserts' using ("serial_number", "part_number", 
  "timestamp", "station_list") as keys.
  
  Parameters
    ----------
    line : string
        Full name of the assembly line
    line_df : dataframe
        The batch dataframe for the given line
    data_source_id : string
        The ID string of the data source 
  """
  
  batch_feature_list = set(getFeatureListFromDF(line_df))
  
  if not batch_feature_list.issubset(lines_feature_dict[line]["curr_feature_list"]):
    lines_feature_dict[line]["prev_feature_list"] = set(map(str, lines_feature_dict[line]["curr_feature_list"]))
    lines_feature_dict[line]["curr_feature_list"] = set(map(str, batch_feature_list.union(lines_feature_dict[line]["curr_feature_list"])))
    lines_feature_dict[line]["next_pivot_schema_version"] = updatePivotSchemaLocal(data_source_id,  
                                                                                   pivot_idx_ddl, 
                                                                                   lines_feature_dict[line]["next_pivot_schema_version"],
                                                                                   lines_feature_dict[line]["curr_feature_list"]
                                                                                  )
    lines_feature_dict[line]["curr_feature_ddl"] = "`" + '` STRING, `'.join(sorted(lines_feature_dict[line]["curr_feature_list"])) + "` STRING"
    
  # the target pivot table
  pivot_out = DeltaTable.forPath(spark, pivotRootDir + f"{line}/")

  # apply current schema to the batch dataframe
  schema_df = (line_df
            .withColumn("feature_json", from_json("feature_json", lines_feature_dict[line]["curr_feature_ddl"]))
            .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", col("feature_json.*"))
           )

  # rows in the pivot table needing to be updated
  if len(pivot_out.toDF().columns) > len(pivot_idx_ddl.split(',')):
    rows_to_update = (schema_df
                      .join(
                        pivot_out.toDF().alias("pivot_tab"), 
                        ["serial_number", "part_number", "timestamp", "station_list", "measurement_date"]
                      )
                      .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", "pivot_tab.*") 
                      .withColumn("feature_json", 
                                  concat(lit('{'), 
                                         concat_ws(',', 
                                                   *[concat(lit('"'), lit(c), lit('":"'), col(c), lit('"')) 
                                                     for c in lines_feature_dict[line]["prev_feature_list"]]
                                                  ),
                                         lit('}')
                                        )
                                 )
                      .withColumn("feature_json", from_json("feature_json", lines_feature_dict[line]["curr_feature_ddl"]))
                      .select("serial_number", "part_number", "timestamp", "station_list", "measurement_date", col("feature_json.*"))
                     )

    # prepare the updates dataframe
    updates_df = (schema_df
                 .union(rows_to_update)
                 .groupBy("serial_number", "part_number", "timestamp", "station_list", "measurement_date")
                 .agg(*[f.max(c).alias(c) for c in lines_feature_dict[line]["curr_feature_list"]])
                )
  else:
    updates_df = schema_df

  # upsert new data into Delta table
  match_on_key = """s.part_number=u.part_number and s.serial_number=u.serial_number and s.timestamp=u.timestamp and s.station_list=u.station_list""" 

  (pivot_out
   .alias("s")
   .merge(updates_df.alias("u"), match_on_key)
   .whenMatchedUpdateAll()
   .whenNotMatchedInsertAll()
   .execute()
  )

In [0]:
#**********************************************************************************************************************
# WRITE PIVOTTED RESULTS TO DELTA TABLE (THE CUBE)
# The pivot table is dynamically populated and updated with transformed streaming data
#**********************************************************************************************************************

lines_feature_dict = {}

def updateSchemaAndDelta(df, batch_id):
  """
  Process the batch dataframe. For each line in the dataframe, update the corresponding delta table.
  
  Parameters
    ----------
    df : dataframe
        The batch dataframe 
    batch_id : string
        The batch ID
  """
  global lines_feature_dict
  
  lines = df.select("line_full_name").distinct().collect()
  for a_row in lines:
    line = a_row["line_full_name"]
    line_df = df.filter(col("line_full_name")==line)
    data_source_id, schema_version =  getDataSourceAndSchemaVersion(line_df)   ## ADD HANDLER for line with multiple schema versions
    
    if not DeltaTable.isDeltaTable(spark, pivotRootDir + f"{line}/"):
      initialize_delta_sink(line)
      
    if not line in lines_feature_dict:
      temp_dict = {}
      temp_dict["curr_feature_list"], \
      temp_dict["next_pivot_schema_version"], \
      temp_dict["curr_feature_ddl"] = load_features_for_line(data_source_id)
      temp_dict["prev_feature_list"] = temp_dict["curr_feature_list"]
      
      lines_feature_dict[line] = temp_dict
      
    
    process_batch_for_line(line, line_df, data_source_id)


savePivot = (pivot_prep_df
             .writeStream
             .queryName("kafka_kv_to_pivot_delta_lake")
             .format("delta")
             .option("checkpointLocation", checkpointPath)
                 .option("mergeSchema", "true")
                 .foreachBatch(updateSchemaAndDelta)
             .outputMode("update")
             .start()
            )

#display(input_df)

In [0]:
display(spark.sql("select * from pivot_schema_table"))

dataSourceId,pivotSchemaVersion,pivotIndexDdl,featureList,updated_on


In [0]:
### For Dev and Testing, load back the table and view
# lines tested: Borg_Warner_XYZ_L14, Dana_Birmingham_ASC1, Dana_Columbia_Loop_1, Dana_Dry_Ridge_700_line
pivot_load_df = spark.read.format('delta').load(pivotRootDir + "Dana_Birmingham_ASC1/")
pivot_load_df.registerTempTable("pivot_results_table")
display(pivot_load_df)
pivot_load_df.count()

part_number,serial_number,timestamp,station_list,measurement_date,00ac6aa6_5d09_4752_a614_bae5db10d432_data_value,01dd9a45_8d4f_4755_8cfe_73c607cae2e1_data_value,0233d4da_4c95_4556_95bf_ea7857f80575_data_value,0459ee74_860f_4f67_b1a4_b79d4f4104ea_data_value,04bb019e_4c7d_4fda_9ace_e031b63d77fb_data_value,05708c2e_a0a1_4550_be84_21e0d2f61806_data_value,05afc17f_f75e_4a5d_aab9_b949658643f5_data_value,05e243f3_a1f7_4810_a162_ec36300f6600_data_value,06dd20be_080e_437b_9a3b_39d6187a561e_data_value,08752ae5_535c_466d_903b_3cd7c770205a_data_value,0a3f0f02_094f_4c28_9f49_2ab56e706e44_data_value,0af234c8_fb62_43fc_a2fa_c44e6f39f989_data_value,0bc6b92d_854b_44e7_97d2_7a129851ec91_data_value,0e32251f_ab4e_4bac_8964_c4414de12ea6_data_value,0eb31e56_26f5_49a3_a005_f9de1dd9ab6a_data_value,0f340bd2_4e64_45b3_bdba_8ac17d6079ad_data_value,10284fdd_650e_460d_8059_bb5883686f7b_data_value,108fbdd9_5564_4563_8947_67bbea7df181_data_value,10fa8668_76da_421e_9d1a_a39d10627ff7_data_value,1121d711_4a10_4ec7_9ad4_d7a13b7fe3b7_data_value,13b9462b_1f91_43ea_8d33_2cf3dc65a740_data_value,1587c59b_c33e_4313_90ba_5711833c291d_data_value,16eb1aad_ffe4_4933_9e1b_73efff35abb2_data_value,16f60585_f542_46a7_b2e1_d93061fb2fb2_data_value,1895fdfb_f244_4069_8efb_ad85b64463e3_data_value,19779d4f_04dc_4617_b524_e85a31996e55_data_value,1aedab30_7758_4bfa_ab0a_0f4d035dc953_data_value,1b1c0137_012a_4153_b9f4_89344fd41369_data_value,1b38eb7b_ee89_48bc_9da2_c8b4a94661fe_data_value,1d20e74b_0eb4_44d5_bb30_e66071139c3e_data_value,1d24d5d9_459f_4947_ab72_b1b66e1a0bdd_data_value,1db9f6b4_4357_4896_9e6d_ace50c99f13b_data_value,1e0db345_ca5e_4ae1_9e6d_561d24b192e7_data_value,1ecf0f53_cdb1_4b64_b9be_a95ba0b18126_data_value,1f842931_5070_453b_a90f_0a0132099f23_data_value,1f89f040_20bc_429b_9aad_9c443fbe9b88_data_value,204d5ba4_3332_4948_a0fa_4ad4d9986f81_data_value,208fd89e_1c51_4891_9fd9_882e0c65046f_data_value,210d6b80_de65_4300_9066_38760f9a5d81_data_value,218aada8_09b3_452a_90a7_c777d9cc9144_data_value,23eb93ef_c427_4a77_910e_8bdc6e2df6d2_data_value,25b8f42a_18dc_4070_8f34_49637fdcc95c_data_value,262130a9_ce3e_4251_9d58_bb66dfc9b4e6_data_value,26b09a25_184b_42fe_85c9_ea9842f97c7b_data_value,27d636a0_1f88_4354_ad6c_5b0501fa40e6_data_value,2a36fb69_dc52_4fdc_b535_6d0c2c00ac39_data_value,2a7d9ae7_b154_424d_8984_4eacc4ec6d47_data_value,2e772fff_8ae3_4f98_bd14_c92cffe3ece5_data_value,2ea7f811_8259_4a98_8e0b_3341127e17a0_data_value,2ffc5a07_5c7f_41bc_8a5a_03c1622157d6_data_value,30a0d2ca_2edd_42e9_af2b_1ba5bf5bcb85_data_value,31b40c13_ab08_4ad8_83c6_f61f1d11d37b_data_value,322356f6_79b5_430d_a30a_64c5d794c5cc_data_value,32c26460_d02e_4411_9d95_a79528b26aec_data_value,3511ae60_16d5_499b_ba4a_df5810322603_data_value,3669f4a9_ff97_4ab0_ac6a_10b35dc2f3e6_data_value,372c900d_bf26_4196_ad8d_329b54306d56_data_value,3d44c6c2_548d_40bd_a3c5_481b5d7b8c37_data_value,3fb731a5_b728_449e_994d_aae90db15770_data_value,40be91a4_0623_4013_9fad_12962cd4d9b0_data_value,40c0f970_9c2f_46c2_b9dd_371c6318db14_data_value,41d12ae9_a6e0_4657_a603_3339f2525dba_data_value,42049724_561b_42dc_ac53_44de2a32e918_data_value,42792d3b_b4d8_46e6_b746_02f9ab38d2c6_data_value,4335f311_91c7_4b75_b17d_dcb868426589_data_value,4348918c_3212_44b2_924a_5ecd8adae59a_data_value,447c6e92_f152_45b3_bff5_474342476330_data_value,451554e7_de5a_47c6_877f_9ceb9456868d_data_value,45f824cd_c6e5_4e53_9734_0feebdaaef54_data_value,475f1267_1573_4d29_9a05_79949823c2ea_data_value,49dcda8c_69be_42d8_acf7_cc893124458b_data_value,4a01f851_d189_4cb0_b8b8_f51b518dd4d9_data_value,4ad9427e_7e96_4da2_aa9d_0aa5e33624fb_data_value,4ce5cd2d_113f_4ffa_8a91_0ac0d4d9af69_data_value,4d68bc0e_a920_4b50_a623_9098a07d2223_data_value,4d7fb731_aa80_4dda_8b0f_11f50a97c917_data_value,53b2ab60_37b6_4048_8615_b7953fe99c4c_data_value,56a0af0b_6db7_438a_9a88_7a2eacf011a2_data_value,5797d135_d65e_4c8a_b470_25d3317f598b_data_value,57b35bd6_3f0f_4c51_9d48_3fc8ee26333b_data_value,592f813b_ad0d_48b9_b732_8e69f975b65e_data_value,5bbaade1_f99e_4152_bc3c_da2e6a4a3211_data_valu

Out[5]: 3174695

In [0]:
%sql
-- code to create and optimize sql table in databricks workspace
drop table if exists pivot_results_table;
create table pivot_results_table
using DELTA
location 'dbfs:/acerta/output/cube/pivot/Borg_Warner_XYZ_L14/';

OPTIMIZE pivot_results_table ZORDER BY (serial_number);

path,metrics
dbfs:/acerta/output/cube/pivot/Borg_Warner_XYZ_L14,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 12, List(minCubeSize(107374182400), List(0, 0), List(12, 79774), 0, List(0, 0), 0, null), 0, 12, 12, false)"


In [0]:
%sql describe history pivot_results_table;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2021-11-22T15:21:02.000+0000,1094680775067414,ljin@acerta.ai,MERGE,"Map(predicate -> (((s.`part_number` = u.`part_number`) AND (s.`serial_number` = u.`serial_number`)) AND ((s.`timestamp` = u.`timestamp`) AND (s.`station_list` = u.`station_list`))), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(1689534426296516),1110-132154-x22gtc30,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 12, executionTimeMs -> 2665, numTargetRowsInserted -> 297, scanTimeMs -> 363, numTargetRowsUpdated -> 0, numOutputRows -> 297, numTargetChangeFilesAdded -> 0, numSourceRows -> 297, numTargetFilesRemoved -> 0, rewriteTimeMs -> 2279)",null
0,2021-11-22T15:20:51.000+0000,1094680775067414,ljin@acerta.ai,WRITE,"Map(mode -> Append, partitionBy -> [""measurement_date""])",null,List(1689534426296516),1110-132154-x22gtc30,null,SnapshotIsolation,true,"Map(numFiles -> 0, numOutputBytes -> 0, numOutputRows -> 0)",null
